In [11]:
import numpy as np
np.random.seed(1234)

In [12]:
num_clients = 10

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

def preprocess_purchase100():
    dataset_path = "data/dataset_purchase"
    with open(dataset_path, "r") as f:
        purchase_dataset = f.readlines()
    x, y = [], []
    for datapoint in purchase_dataset:
        split = datapoint.rstrip().split(",")
        label = int(split[0]) - 1  # The first value is the label
        features = np.array(split[1:], dtype=np.float32)  # The next values are the features
        x.append(features)
        y.append(label)

    x = np.array(x, dtype=np.float32)
    y = np.array(y, dtype=np.int8)

    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.1, random_state=1234)

    y_train_onehot = OneHotEncoder(sparse=False).fit_transform(np.expand_dims(y_train, axis=1))
    y_test_onehot = OneHotEncoder(sparse=False).fit_transform(np.expand_dims(y_test, axis=1))

    x_test = np.array(x_test,dtype=np.float32)
    y_test_onehot = np.array(y_test_onehot,dtype=np.float32)

    xs,ys = [[] for _ in range(num_clients)],[[] for _ in range(num_clients)]
    for index in range(x_train.shape[0]):
        xs[y_train[index]%num_clients].append(x_train[index])
        ys[y_train[index]%num_clients].append(y_train_onehot[index])

    train_datasets_list = []
    for x_i,y_i in zip(xs,ys):
        x_i = np.array(x_i, dtype=np.float32)
        y_i = np.array(y_i, dtype=np.float32)
        train_datasets_list.append([x_i,y_i,x_test,y_test_onehot])
    
    return train_datasets_list

In [14]:
train_datasets_list = preprocess_purchase100()

In [15]:
import os
import numpy as np

if not os.path.exists("./datasets"):
    os.mkdir("./datasets")
if not os.path.exists("./datasets/train"):
    os.mkdir("./datasets/train")
    
for i,dataset in enumerate(train_datasets_list):
    if not os.path.exists("./datasets/train/%d"%(i)):
        os.mkdir("./datasets/train/%d"%(i))
    np.save("./datasets/train/%d/train_x.npy"%(i),dataset[0])
    np.save("./datasets/train/%d/train_y.npy"%(i),dataset[1])
    np.save("./datasets/train/%d/test_x.npy"%(i),dataset[2])
    np.save("./datasets/train/%d/test_y.npy"%(i),dataset[3])